# 𝕃𝕠𝕤𝕤 𝕗𝕦𝕟𝕔𝕥𝕚𝕠𝕟𝕤 𝕗𝕠𝕣 𝕤𝕖𝕞𝕒𝕟𝕥𝕚𝕔 𝕤𝕖𝕘𝕞𝕖𝕟𝕥𝕒𝕥𝕚𝕠𝕟
* A survey of loss functions for semantic segmentation: https://arxiv.org/pdf/2006.14822.pdf
* Loss functions for image segmentation: https://github.com/JunMa11/SegLoss

In [1]:
# unit test
import unittest
# test data
import medmnist
from medmnist import INFO

from PIL import Image, ImageDraw

# "helper" needs to be part of sys path
import sys
sys.path.insert(0, "helper")
sys.path.insert(0, "/helper")
sys.path.insert(0, "./helper")
sys.path.insert(0, "../helper")
print(sys.path)

%load_ext autoreload
%autoreload 2

# own module
from compute.loss import * 

['../helper', './helper', '/helper', 'helper', 'C:\\Users\\Prinzessin\\projects\\decentnet\\datasceyence\\examples', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\python39.zip', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\DLLs', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta', '', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Prinzessin\\.ipython']


In [2]:
class UnitTestLoss(unittest.TestCase):
    
    def test_shape(self):
        
        criterion = ShapeLoss()
        
        # key : batch, classes, dim1 (image), dim2 (image)
        # torch.Size([1, 2, 3, 3])
        model_outputs = {"good - A regular":
                            torch.tensor([[ # good
                                    [[-0.4, 0.3, 0.5], [-0.31, -0.1, 0.4], [-0.5, -0.4, 0.9]],
                                    [[0.5, -0.5, -0.3], [0.7, 0.8, 0.1], [0.31, 0.4, -0.1]],
                                    ]]),
                         "okay - B regular":
                         torch.tensor([[ # middle
                                    [[0.4, 0.3, -0.2], [-0.31, 0.3, 0.4], [0.5, -0.4, 0.9]],
                                    [[-0.5, -0.5, 0.3], [0.7, -0.2, 0.1], [-0.31, 0.4, -0.1]],
                                    ]]),
                         "poor - A inverse":
                         torch.tensor([[ # bad
                                    [[0.5, -0.5, -0.3], [0.7, 0.8, 0.1], [0.31, 0.4, -0.1]],
                                    [[-0.4, 0.3, 0.5], [-0.31, -0.1, 0.4], [-0.5, -0.4, 0.9]],
                                    ]])            
                        }

        # batch, dim1 (image), dim2 (image)
        # torch.Size([1, 3, 3])
        ground_truth = torch.tensor([[
                                    [1, 0, 0], [1, 1, 0], [1, 1, 0],
                                    ]])
        
        
        
        print('='*50)
        print("AAAI_sdf_loss = shape based loss")
        
        for key, model_output in model_outputs.items():
            loss = criterion(model_output=model_output, ground_truth=ground_truth)
            print(key, "loss:", round(loss.item(), 3))
        
    def test_iou(self):
        
        criterion = IoULoss(mode="multiclass", classes=[0,1], log_loss=True)
        
        # key : batch, classes, dim1 (image), dim2 (image)
        # torch.Size([1, 2, 3, 3])
        model_outputs = {"good":
                            torch.tensor([[ # good
                                    [[-0.4, 0.3, 0.5], [-0.31, -0.1, 0.4], [-0.5, -0.4, 0.9]],
                                    [[0.5, -0.5, -0.3], [0.7, 0.8, 0.1], [0.31, 0.4, -0.1]],
                                    ]]),
                         "okay":
                         torch.tensor([[ # middle
                                    [[0.4, 0.3, -0.2], [-0.31, 0.3, 0.4], [0.5, -0.4, 0.9]],
                                    [[-0.5, -0.5, 0.3], [0.7, -0.2, 0.1], [-0.31, 0.4, -0.1]],
                                    ]]),
                         "poor":
                         torch.tensor([[ # bad
                                    [[0.5, -0.5, -0.3], [0.7, 0.8, 0.1], [0.31, 0.4, -0.1]],
                                    [[-0.4, 0.3, 0.5], [-0.31, -0.1, 0.4], [-0.5, -0.4, 0.9]],
                                    ]])            
                        }
        
        # batch, dim1 (image), dim2 (image)
        # torch.Size([1, 3, 3])
        ground_truth = torch.tensor([[
                                    [1, 0, 0], [1, 1, 0], [1, 1, 0],
                                    ]])
        
        print('='*50)
        print("IoU")
        
        for key, model_output in model_outputs.items():
            loss = criterion(model_output=model_output, ground_truth=ground_truth)
            print(key, "loss:", round(loss.item(), 3))
        
    def test_dice(self):
        
        criterion = DiceLoss(mode="multiclass", classes=[0,1])
        
        # key : batch, classes, dim1 (image), dim2 (image)
        # torch.Size([1, 2, 3, 3])
        model_outputs = {"good":
                            torch.tensor([[ # good
                                    [[-0.4, 0.3, 0.5], [-0.31, -0.1, 0.4], [-0.5, -0.4, 0.9]],
                                    [[0.5, -0.5, -0.3], [0.7, 0.8, 0.1], [0.31, 0.4, -0.1]],
                                    ]]),
                         "okay":
                         torch.tensor([[ # middle
                                    [[0.4, 0.3, -0.2], [-0.31, 0.3, 0.4], [0.5, -0.4, 0.9]],
                                    [[-0.5, -0.5, 0.3], [0.7, -0.2, 0.1], [-0.31, 0.4, -0.1]],
                                    ]]),
                         "poor":
                         torch.tensor([[ # bad
                                    [[0.5, -0.5, -0.3], [0.7, 0.8, 0.1], [0.31, 0.4, -0.1]],
                                    [[-0.4, 0.3, 0.5], [-0.31, -0.1, 0.4], [-0.5, -0.4, 0.9]],
                                    ]])            
                        }
        
        # batch, dim1 (image), dim2 (image)
        # torch.Size([1, 3, 3])
        ground_truth = torch.tensor([[
                                    [1, 0, 0], [1, 1, 0], [1, 1, 0],
                                    ]])
        
        print('='*50)
        print("Dice")
        
        for key, model_output in model_outputs.items():
            loss = criterion(model_output=model_output, ground_truth=ground_truth)
            print(key, "loss:", round(loss.item(), 3))
            
            
    def test_cross_entropy(self):
        
        # Example of target with class indices
        criterion = torch.nn.CrossEntropyLoss()


        # key : batch, classes, dim1 (image), dim2 (image)
        # torch.Size([1, 2, 3, 3])
        model_outputs = {"good":
                            torch.tensor([[ # good
                                    [[-0.4, 0.3, 0.5], [-0.31, -0.1, 0.4], [-0.5, -0.4, 0.9]],
                                    [[0.5, -0.5, -0.3], [0.7, 0.8, 0.1], [0.31, 0.4, -0.1]],
                                    ]]),
                         "okay":
                         torch.tensor([[ # middle
                                    [[0.4, 0.3, -0.2], [-0.31, 0.3, 0.4], [0.5, -0.4, 0.9]],
                                    [[-0.5, -0.5, 0.3], [0.7, -0.2, 0.1], [-0.31, 0.4, -0.1]],
                                    ]]),
                         "poor":
                         torch.tensor([[ # bad
                                    [[0.5, -0.5, -0.3], [0.7, 0.8, 0.1], [0.31, 0.4, -0.1]],
                                    [[-0.4, 0.3, 0.5], [-0.31, -0.1, 0.4], [-0.5, -0.4, 0.9]],
                                    ]])            
                        }

        

        # batch, dim1 (image), dim2 (image)
        # torch.Size([1, 3, 3])
        ground_truth = torch.tensor([[
                            [1, 0, 0], [1, 1, 0], [1, 1, 0],
                            ]])
        

        print('='*50)
        print("Cross Entropy Loss")

        for key, model_output in model_outputs.items():
            loss = criterion(model_output, ground_truth)
            if False:
                print(model_output)
                print(model_outputs["good"].shape) 
                print(ground_truth)
                print(ground_truth.shape) 
            print(key, "loss:", round(loss.item(), 3))
            

In [3]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.139s

OK


Cross Entropy Loss
good loss: 0.371
okay loss: 0.699
poor loss: 1.185
Dice
good loss: 0.311
okay loss: 0.47
poor loss: 0.692
IoU
good loss: 0.645
okay loss: 1.02
poor loss: 1.703
AAAI_sdf_loss = shape based loss
good - A regular loss: 0.572
okay - B regular loss: 0.61
poor - A inverse loss: 0.572
